# Packages

In [1]:
# Data Processing and Analytics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
# Machine Learning and ML Preprocessing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
# Neural Networks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# Presets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
%matplotlib inline

## Set Seed

In [2]:
np.random.seed(100)

# Import Data

In [3]:
data = pd.read_csv('data_for_ml.csv')
data.head(2)

,INDEX_,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,3366652,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major
1,3370334,19,weekday,Rain,Dark,Wet,4.8,1.00,Midblock,Traffic Signal including Transit,Major Arterial,60,351.0,63.0,Residential,452,205,183,Over 65,Normal,Crossing without ROW,automobile,speeding,Major


In [4]:
df = data.drop('INDEX_', axis=1)
df.head(1)

,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major


In [5]:
# Integer encode the INJURY column
injury_map = {
    'Fatal' : 1,
    'Major' : 0,
    'Minor' : 0,
    'Minimal' : 0,
    np.nan : 0
}
# 1 means its Fatal and 0 means non-fatal
df['INJURY'] = df['INJURY'].replace(injury_map).astype(int)
df.head(1)

,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,0


# ML Data Preprocessing

In [6]:
# Split into X and y variables
X = df.iloc[:, :-1].values
y = df.iloc[:,-1].values

In [7]:
# One hot encode the categorical columns
cat_columns_index = [1, 2, 3, 4, 7, 8, 9, 13, 17, 18, 19, 20, 21]
encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X[:, cat_columns_index])

In [8]:
# Combine the categorical and numerical columns
num_columns_index = [0, 5, 6, 10, 11, 12, 14, 15, 16]
X_encoded = np.concatenate((X_cat, X[:, num_columns_index].astype(float)), axis=1)
print(X_encoded)

[[1.000e+00 0.000e+00 0.000e+00 ... 5.040e+02 2.630e+02 2.250e+02]
 [1.000e+00 0.000e+00 0.000e+00 ... 4.520e+02 2.050e+02 1.830e+02]
 [1.000e+00 0.000e+00 1.000e+00 ... 8.070e+02 3.750e+02 3.220e+02]
 ...
 [1.000e+00 0.000e+00 0.000e+00 ... 4.820e+02 2.520e+02 2.310e+02]
 [0.000e+00 1.000e+00 1.000e+00 ... 1.981e+03 1.195e+03 1.112e+03]
 [1.000e+00 0.000e+00 1.000e+00 ... 1.170e+03 4.820e+02 4.670e+02]]


In [9]:
# Split encoded X and y to train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.20, random_state=100)

In [10]:
# Scale the numerical features
scaler = StandardScaler()
X_train[:, -len(num_columns_index):] = scaler.fit_transform(X_train[:, -len(num_columns_index):])
X_test[:, -len(num_columns_index):] = scaler.transform(X_test[:, -len(num_columns_index):])
print(f"X_train dimension: {X_train.shape}")
print(f"y_train dimension: {y_train.shape}")
print(f"X_test dimension: {X_test.shape}")
print(f"y_test dimension: {y_test.shape}")

X_train dimension: (2490, 83)
y_train dimension: (2490,)
X_test dimension: (623, 83)
y_test dimension: (623,)


# ML Modeling and Metrics

In [11]:
models = [KNeighborsClassifier(), RandomForestClassifier(), XGBClassifier()]

In [12]:
# Define parameter grids
param_grid_knn = {
    'n_neighbors' : [i for i in range(1,36)],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan']
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],                
    'max_features': ['auto', 'sqrt', 'log2'],      
    'max_depth': [None, 10, 20, 30],               
    'min_samples_split': [2, 5, 10],               
    'min_samples_leaf': [1, 2, 4],                 
    'bootstrap': [True, False],                    
    'criterion': ['gini', 'entropy']               
}

param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0]
}

param_grids = [param_grid_knn, param_grid_rf, param_grid_xgb]

In [13]:
# Iterate on both grids and models to fit the model with our training data
for model, grid in zip(models, param_grids):
    random_search = RandomizedSearchCV(estimator=model,
                                      param_distributions=grid,
                                      n_iter=100,
                                      cv=5,
                                      scoring='accuracy',
                                      random_state=100)
    random_search.fit(X_train, y_train)
    y_pred = random_search.predict(X_test)
    training_accuracy = random_search.best_score_
    best_parameters = random_search.best_params_
    print(f'{model} Modeling Results:\n')
    print('Best Parameters:')
    print(f'{best_parameters}\n')
    print(f"Training Accuracy: {training_accuracy*100:.2f}%")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')*100:.2f}%\n")
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:")
    print(f"{cm}\n")

KNeighborsClassifier() Modeling Results:

Best Parameters:
{'weights': 'uniform', 'n_neighbors': 12, 'metric': 'manhattan'}

Training Accuracy: 83.90%
Testing Accuracy: 84.59%
Precision: 79.42%
Recall: 84.59%
F1 Score: 79.30%

Confusion Matrix:
[[521   7]
 [ 89   6]]

RandomForestClassifier() Modeling Results:

Best Parameters:
{'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': True}

Training Accuracy: 83.45%
Testing Accuracy: 85.39%
Precision: 82.59%
Recall: 85.39%
F1 Score: 80.49%

Confusion Matrix:
[[523   5]
 [ 86   9]]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_con

# Neural Network Modeling

In [14]:
tf.random.set_seed(100)

## Design 1 - 2 Hidden Layers

In [15]:
# Construct NN Architecture
design1 = Sequential([
    Dense(units=50, activation='relu', input_shape=(83,)),
    Dropout(rate=0.4),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design1.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design1.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 4ms/step - loss: 0.4832 - accuracy: 0.8325 - val_loss: 0.4120 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4236 - accuracy: 0.8337 - val_loss: 0.4003 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4093 - accuracy: 0.8361 - val_loss: 0.4037 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4088 - accuracy: 0.8357 - val_loss: 0.3957 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4018 - accuracy: 0.8349 - val_loss: 0.3980 - val_accuracy: 0.8459
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4020 - accuracy: 0.8357 - val_loss: 0.4238 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4121 - accuracy: 0.8369 - val_loss: 0.3960 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3524 - accuracy: 0.8550 - val_loss: 0.4191 - val_accuracy: 0.8347
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3537 - accuracy: 0.8570 - val_loss: 0.4117 - val_accuracy: 0.8363
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3504 - accuracy: 0.8562 - val_loss: 0.4128 - val_accuracy: 0.8411
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3467 - accuracy: 0.8614 - val_loss: 0.4004 - val_accuracy: 0.8475
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3549 - accuracy: 0.8522 - val_loss: 0.4174 - val_accuracy: 0.8379
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3472 - accuracy: 0.8602 - val_loss: 0.4027 - val_accuracy: 0.8427
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3468 - accuracy: 0.8582 - val_loss: 0.4061 - val_accuracy: 0.8443
Epoch 

## Design 2 - 4 Hidden Layers

In [16]:
# Construct NN Architecture
design2 = Sequential([
    Dense(units=50, activation='relu', input_shape=(83,)),
    Dropout(rate=0.4),
    Dense(units=50, activation='relu'),
    Dropout(rate=0.4),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design2.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design2.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 4ms/step - loss: 0.5158 - accuracy: 0.8253 - val_loss: 0.4109 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4313 - accuracy: 0.8337 - val_loss: 0.3885 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4142 - accuracy: 0.8337 - val_loss: 0.3946 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4090 - accuracy: 0.8337 - val_loss: 0.3853 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3993 - accuracy: 0.8337 - val_loss: 0.3891 - val_accuracy: 0.8475
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3954 - accuracy: 0.8329 - val_loss: 0.4056 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.3925 - accuracy: 0.8333 - val_loss: 0.3853 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2447 - accuracy: 0.8839 - val_loss: 0.5545 - val_accuracy: 0.8411
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2334 - accuracy: 0.8904 - val_loss: 0.5987 - val_accuracy: 0.8411
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2406 - accuracy: 0.8888 - val_loss: 0.6241 - val_accuracy: 0.8363
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2346 - accuracy: 0.8876 - val_loss: 0.5698 - val_accuracy: 0.8379
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2316 - accuracy: 0.8884 - val_loss: 0.6321 - val_accuracy: 0.8411
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2352 - accuracy: 0.8835 - val_loss: 0.6105 - val_accuracy: 0.8507
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.2279 - accuracy: 0.8932 - val_loss: 0.6467 - val_accuracy: 0.8491
Epoch 

## Design 3 - 6 Hidden Layers

In [17]:
# Construct NN Architecture
design3 = Sequential([
    Dense(units=50, activation='relu', input_shape=(83,)),
    Dropout(rate=0.4),
    Dense(units=50, activation='relu'),
    Dropout(rate=0.4),
    Dense(units=50, activation='relu'),
    Dropout(rate=0.4),
    Dense(units=1, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design3.compile(loss=BinaryCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design3.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 4ms/step - loss: 0.5098 - accuracy: 0.8333 - val_loss: 0.4115 - val_accuracy: 0.8475
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4389 - accuracy: 0.8337 - val_loss: 0.3976 - val_accuracy: 0.8475
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4297 - accuracy: 0.8337 - val_loss: 0.4205 - val_accuracy: 0.8475
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4270 - accuracy: 0.8337 - val_loss: 0.3846 - val_accuracy: 0.8475
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4087 - accuracy: 0.8337 - val_loss: 0.3968 - val_accuracy: 0.8475
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4045 - accuracy: 0.8353 - val_loss: 0.4059 - val_accuracy: 0.8475
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.4032 - accuracy: 0.8382 - val_loss: 0.3836 - val_accuracy: 0.8475
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2676 - accuracy: 0.8831 - val_loss: 0.5055 - val_accuracy: 0.8331
Epoch 60/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2726 - accuracy: 0.8739 - val_loss: 0.6561 - val_accuracy: 0.8315
Epoch 61/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2585 - accuracy: 0.8884 - val_loss: 0.6465 - val_accuracy: 0.8427
Epoch 62/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2672 - accuracy: 0.8863 - val_loss: 0.5375 - val_accuracy: 0.8395
Epoch 63/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2757 - accuracy: 0.8811 - val_loss: 0.5768 - val_accuracy: 0.8411
Epoch 64/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2848 - accuracy: 0.8827 - val_loss: 0.6036 - val_accuracy: 0.8411
Epoch 65/100
78/78 [==============================] - 0s 3ms/step - loss: 0.2701 - accuracy: 0.8900 - val_loss: 0.7712 - val_accuracy: 0.8379
Epoch 

# NN Performance Metrics

In [18]:
models = [design1, design2, design3]
for model in models:
    y_pred = model.predict(X_test)
    y_pred_classes = (y_pred > 0.5).astype(int)
    train_loss, train_accuracy = model.evaluate(X_train, y_train)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    print(f"Model Performance Metrics:")
    print(f"Training Accuracy: {train_accuracy*100:.2f}%")
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"F1 Score: {f1*100:.2f}%\n")
    print(f"Confusion Matrix:")
    print(f"{confusion_matrix(y_test, y_pred_classes)}\n")

20/20 [==============================] - 0s 1ms/step - loss: 0.4213 - accuracy: 0.8411
Model Performance Metrics:
Training Accuracy: 87.43%
Test Accuracy: 84.11%
Precision: 78.86%
Recall: 84.11%
F1 Score: 79.67%

Confusion Matrix:
[[515  13]
 [ 86   9]]

20/20 [==============================] - 0s 1ms/step - loss: 0.6570 - accuracy: 0.8202
Model Performance Metrics:
Training Accuracy: 93.82%
Test Accuracy: 82.02%
Precision: 79.06%
Recall: 82.02%
F1 Score: 80.22%

Confusion Matrix:
[[490  38]
 [ 74  21]]

20/20 [==============================] - 0s 1ms/step - loss: 0.8309 - accuracy: 0.8427
Model Performance Metrics:
Training Accuracy: 91.57%
Test Accuracy: 84.27%
Precision: 79.73%
Recall: 84.27%
F1 Score: 80.36%

Confusion Matrix:
[[513  15]
 [ 83  12]]

